# Week 8 Study Notebook

# UC San Diego OMDS DSC 207

## Imports

In [7]:
#LOAD LIBRARIES
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import os

# Part 1: Clustering

#### 1.1 For our KMeans clustering task, read only the following columns into a dataframe `X` using the following link: https://gist.githubusercontent.com/netj/8836201/raw/.

1. "sepal.length"
2. "sepal.width"
3. "petal.length"
4. "petal.width"

Print the head() of `X`. [1 pt]

In [8]:
# set the working directory
os.chdir('/home/scotty/dsc_207_week8')
X = pd.read_csv('week8.csv')
print(X.head())

   sepal.length  sepal.width  petal.length  petal.width variety
0           5.1          3.5           1.4          0.2  Setosa
1           4.9          3.0           1.4          0.2  Setosa
2           4.7          3.2           1.3          0.2  Setosa
3           4.6          3.1           1.5          0.2  Setosa
4           5.0          3.6           1.4          0.2  Setosa


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

#### 1.2 Defining the model

Import and define a model using <a href='http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html'>sklearn.cluster.kmeans</a> [0.5 pt].<br>
**Pick any value for the number of clusters for now.**<br>


In [ ]:

kmeans = ...

#### 1.3 [Fit and predict](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans.fit_predict) the kmeans model on `X`. Print the predicted values. [0.5 pt]

In [ ]:

y_kmeans = ...

#### 1.4 Print the center points of the clusters that the model created. (Hint - look into the documentation to see how we can easily find it) [0.5 pt]

#### 1.5 Find and print the [silhouette score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html) of the the data using the current model. [0.5 pt]

<!-- END QUESTION -->

### Visualization

In [ ]:
# Run this cell to see clusters being formed for pairs of features. Make sure to have as many colors as k and you have used similar naming of variables (update the cell below if not).
plt.scatter(data=X, x='sepal.length', y='sepal.width')
colors = ['r', 'y', 'b', 'c', 'm', 'g', 'k', 'w','orange']
for i, coor in enumerate(kmeans.cluster_centers_):
  plt.scatter(coor[0], coor[1], s=200, c=colors[i], marker='s')
plt.show()

### Experimentation

We chose an arbitrary value of k for the above experiment. In the lecture videos, we saw that elbow method is one of the ways to make a better guess for the value of k, although not always useful.

In [ ]:
#Run this cell. Make sure you update the variable names if required.
from sklearn.cluster import KMeans
wcss = []

for i in range(1, 10): #Specify for which values of k you want to try
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)

In [ ]:
#Run this cell
plt.plot(range(1, 10), wcss) #Specify for which values of k you want to try
plt.title('Elbow method - Line chart')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS') #within cluster sum of squares
plt.show()

#### 1.6 Based on the videos, what can you infer from the elbow graph created? what should be a good value of k (number of clusters)? [0.5 pt]

_Type your answer here, replacing this text._

# Part 2: Neural Networks


For this section, we will be working with the CIFAR-10 dataset.

The CIFAR-10 dataset consists of 32x32 color images, with the 10 classes below:
* airplane
* automobile
* bird
* cat
* deer
* dog
* frog
* horse
* ship
* truck


The cells below will load and visualize the data for you.

In [ ]:
# Load libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

**Note**: The datset might take ~5 mins to load

In [ ]:
# Load the CIFAR dataset
cifar = fetch_openml('cifar_10_small', version=1)
X, y = cifar.data, cifar.target

print(X.shape)
print(y.shape)

In [ ]:
# Convert input features and labels to NumPy arrays so that data "plays nice" with libraries
X = np.array(X)
y = np.array(y)

The cell below visualizes the images.

In [ ]:
def visualize_images(X, y, num_rows=2, num_cols=5):
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 6))

    # Plot each image
    for i, ax in enumerate(axes.ravel()):
        if i < len(X):

            # Creating the image
            img = np.stack([X[i].reshape(3,32, 32)[0], X[i].reshape(3,32, 32)[1], X[i].reshape(3,32, 32)[2]], axis=2)

            # Display the image
            ax.imshow(img)
            label = str(int(float(y[i])))

            ax.axis('off')

    plt.tight_layout()
    plt.show()

# Visualize first 10 images
visualize_images(X, y)

#### 2.1 Split the data into training and testing sets. [0.5 pts]

In [ ]:
X_train, X_test, y_train, y_test = ...

#### 2.2 Scale the data using a StandardScaler. [0.5 pts]

In [ ]:
scaler = StandardScaler()
X_train = ...
X_test = ...

#### 2.3 Initialize and train an MLPClassifier model. [2 pts]

Use the following hyperparameters:
* Hidden layer sizes: 256, 128, 64
* Maximum iterations: 50

Set verbose=True when creating the MLPClassifier to see which iteration of training it is on.
This may take around 5 minutes to run.

In [ ]:
mlp = ...

#### 2.4 Evaluate and print the the accuracy of the model. [1 pt]

In [ ]:
accuracy = ...

# Part 3: Video (Optional)

Tune the hyperparameters of your Multi-layer Perceptron classifier to improve your accuracy from part 2.4. Describe your strategy for doing so.

LINK TO VIDEO HERE: